## Installs and Imports

In [2]:
!pip install open3d

In [2]:
import numpy as np
import open3d as o3d

## Load Files

In [13]:
print("Load a ply point cloud, print it, and render it")
pcd = o3d.io.read_point_cloud("../plyfolder/points3.ply")
print(pcd)
print(np.asarray(pcd.points))
o3d.visualization.draw_geometries([pcd])

Load a ply point cloud, print it, and render it
[Open3D DEBUG] Format auto File plyfolder/points3.ply
[Open3D DEBUG] Read geometry::PointCloud: 25508 vertices.
[Open3D DEBUG] [RemoveNonFinitePoints] 0 nan points have been removed.
PointCloud with 25508 points.
[[-8.854995 -8.854995 45.178547]
 [-8.739034 -8.849655 45.151302]
 [-8.617654 -8.838619 45.094997]
 ...
 [10.059387 10.32067  53.323016]
 [10.180428 10.310946 53.272779]
 [10.32945  10.32945  53.368382]]
[Open3D DEBUG] Add geometry and update bounding box to [(-10.7799, -8.8968, 36.8682) - (10.6456, 10.6452, 55.0015)]


## Outlier Removal

In [22]:
downpcd = pcd.voxel_down_sample(voxel_size=0.1)

In [23]:
o3d.visualization.draw_geometries([downpcd])

## Mesh Convertion

In [26]:
alpha = 0.7

mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(downpcd, alpha)
mesh.compute_vertex_normals()
print(mesh)

TriangleMesh with 21367 points and 51648 triangles.


In [28]:
o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

## Stitching

In [16]:
# http://www.open3d.org/docs/0.7.0/tutorial/Advanced/multiway_registration.html
voxel_size = 0.1
max_correspondence_distance_coarse = voxel_size * 15
max_correspondence_distance_fine = voxel_size * 1.5

file_paths = ["plyfolder/points0.ply","plyfolder/points1.ply","plyfolder/points2.ply"]
def load_point_clouds(voxel_size=0.0):
    pcds = []
    for i in range(3):
        pcd = o3d.io.read_point_cloud(file_paths[i])
        pcd_down = pcd.voxel_down_sample(voxel_size=voxel_size)
        pcds.append(pcd_down)
    return pcds


def pairwise_registration(source, target):
    print("Apply point-to-plane ICP")
    icp_coarse = o3d.registration.registration_icp(
        source, target, max_correspondence_distance_coarse, np.identity(4),
        o3d.registration.TransformationEstimationPointToPlane())
    icp_fine = o3d.registration.registration_icp(
        source, target, max_correspondence_distance_fine,
        icp_coarse.transformation,
        o3d.registration.TransformationEstimationPointToPlane())
    transformation_icp = icp_fine.transformation
    information_icp = o3d.registration.get_information_matrix_from_point_clouds(
        source, target, max_correspondence_distance_fine,
        icp_fine.transformation)
    return transformation_icp, information_icp


def full_registration(pcds, max_correspondence_distance_coarse,
                      max_correspondence_distance_fine):
    pose_graph = o3d.registration.PoseGraph()
    odometry = np.identity(4)
    pose_graph.nodes.append(o3d.registration.PoseGraphNode(odometry))
    n_pcds = len(pcds)
    for source_id in range(n_pcds):
        for target_id in range(source_id + 1, n_pcds):
            transformation_icp, information_icp = pairwise_registration(
                pcds[source_id], pcds[target_id])
            print("Build o3d.registration.PoseGraph")
            if target_id == source_id + 1:  # odometry case
                odometry = np.dot(transformation_icp, odometry)
                pose_graph.nodes.append(
                    o3d.registration.PoseGraphNode(np.linalg.inv(odometry)))
                pose_graph.edges.append(
                    o3d.registration.PoseGraphEdge(source_id,
                                                   target_id,
                                                   transformation_icp,
                                                   information_icp,
                                                   uncertain=False))
            else:  # loop closure case
                pose_graph.edges.append(
                    o3d.registration.PoseGraphEdge(source_id,
                                                   target_id,
                                                   transformation_icp,
                                                   information_icp,
                                                   uncertain=True))
    return pose_graph


o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Debug)
pcds_down = load_point_clouds(voxel_size)
o3d.visualization.draw_geometries(pcds_down)

print("Full registration ...")
pose_graph = full_registration(pcds_down,
                               max_correspondence_distance_coarse,
                               max_correspondence_distance_fine)

print("Optimizing PoseGraph ...")
option = o3d.registration.GlobalOptimizationOption(
    max_correspondence_distance=max_correspondence_distance_fine,
    edge_prune_threshold=0.25,
    reference_node=0)
o3d.registration.global_optimization(
    pose_graph, o3d.registration.GlobalOptimizationLevenbergMarquardt(),
    o3d.registration.GlobalOptimizationConvergenceCriteria(), option)

print("Transform points and display")
for point_id in range(len(pcds_down)):
    print(pose_graph.nodes[point_id].pose)
    pcds_down[point_id].transform(pose_graph.nodes[point_id].pose)
o3d.visualization.draw_geometries(pcds_down)

print("Make a combined point cloud")
pcds = load_point_clouds(voxel_size)
pcd_combined = o3d.geometry.PointCloud()
for point_id in range(len(pcds)):
    pcds[point_id].transform(pose_graph.nodes[point_id].pose)
    pcd_combined += pcds[point_id]
pcd_combined_down = o3d.geometry.voxel_down_sample(pcd_combined,
                                                       voxel_size=voxel_size)
o3d.io.write_point_cloud("multiway_registration.pcd", pcd_combined_down)
o3d.visualization.draw_geometries([pcd_combined_down])

[Open3D DEBUG] Format auto File plyfolder/points0.ply
[Open3D DEBUG] Read geometry::PointCloud: 24775 vertices.
[Open3D DEBUG] [RemoveNonFinitePoints] 0 nan points have been removed.
[Open3D DEBUG] Pointcloud down sampled from 24775 points to 24760 points.
[Open3D DEBUG] Format auto File plyfolder/points1.ply
[Open3D DEBUG] Read geometry::PointCloud: 25350 vertices.
[Open3D DEBUG] [RemoveNonFinitePoints] 0 nan points have been removed.
[Open3D DEBUG] Pointcloud down sampled from 25350 points to 25338 points.
[Open3D DEBUG] Format auto File plyfolder/points2.ply
[Open3D DEBUG] Read geometry::PointCloud: 25371 vertices.
[Open3D DEBUG] [RemoveNonFinitePoints] 0 nan points have been removed.
[Open3D DEBUG] Pointcloud down sampled from 25371 points to 25350 points.
[Open3D DEBUG] Add geometry and update bounding box to [(-10.7788, -10.2105, 38.6302) - (10.6511, 10.6452, 55.0300)]
[Open3D DEBUG] Add geometry and update bounding box to [(-10.7788, -10.2105, 37.8673) - (10.6511, 10.6452, 55.03

AttributeError: module 'open3d' has no attribute 'registration'